In [1]:
print('google net')

google net


In [2]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10


In [3]:
# 定义一个卷积加一个Relu，一个batchnorm
def conv_relu(in_channel, out_channel, kernel, stride=1, padding=0):
    layer = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel, stride, padding),
        nn.BatchNorm2d(out_channel, eps=1e-3),
        nn.ReLU(True)
    )
    return layer



In [4]:
class inception(nn.Module):
    def __init__(self, in_channel, out1_1, out2_1, out2_3, out3_1, out3_5, out4_1):
        super(inception, self).__init__()
        # 第一条线路
        self.brach1x1 = conv_relu(in_channel, out1_1, 1)
        
        # 第二条线路
        self.branch3x3 = nn.Sequential(
            conv_relu(in_channel, out2_1, 1),
            conv_relu(out2_1, out2_3, 3, padding=1)
        )
        
        # 第三条线路
        self.branch5x5 = nn.Sequential(
            conv_relu(in_channel, out3_1, 1),
            conv_relu(out3_1, out3_5, 5, padding=2)
        )
        
        # 第四条线路
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            conv_relu(in_channel, out4_1, 1)
        )
    
    def forward(self, x):
        f1 = self.brach1x1(x)
        f2 = self.branch3x3(x)
        f3 = self.branch5x5(x)
        f4 = self.branch_pool(x)
        # print('f1:{}, \nf2:{}, \nf3:{}, \nf4:{}'.format(f1.shape, f2.shape, f3.shape, f4.shape))
        output = torch.cat((f1, f2, f3, f4), dim=1)
        # print('output.shape:{}'.format(output.shape))
        return output
    

        

In [5]:
test_net = inception(3, 64, 48, 64, 64, 96, 32)
test_x = Variable(torch.zeros(1, 3, 96, 96))
print('input shape: {}x{}x{}'.format(test_x.shape[1], test_x.shape[2], test_x.shape[3]))
test_y = test_net(test_x)
print('output shape: {}x{}x{}'.format(test_y.shape[1], test_y.shape[2], test_y.shape[3]))

input shape: 3x96x96
output shape: 256x96x96


In [6]:
# 定义只有一个输出的googlenet
class googlenet(nn.Module):
    def __init__(self, in_channel, num_classes, verbose=False):
        super(googlenet, self).__init__()
        self.verbose = verbose
        
        self.block1 = nn.Sequential(
            conv_relu(in_channel, out_channel=64, kernel=7, stride=2, padding=3),
            nn.MaxPool2d(3, 2)
        )
        
        self.block2 = nn.Sequential(
            conv_relu(64, 64, kernel=1),
            conv_relu(64, 192, kernel=3, padding=1),
            nn.MaxPool2d(3, 2)
        )
        
        self.block3 = nn.Sequential(
            inception(192, 64, 96, 128, 16, 32, 32),
            inception(256, 128, 128, 192, 32, 96, 64),
            nn.MaxPool2d(3, 2)
        )
        
        self.block4 = nn.Sequential(
            inception(480, 192, 96, 208, 16, 48, 64),
            inception(512, 160, 112, 224, 24, 64, 64),
            inception(512, 128, 128, 256, 24, 64, 64),
            inception(512, 112, 144, 288, 32, 64, 64),
            inception(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(3, 2)
        )
        
        self.block5 = nn.Sequential(
            inception(832,256,160,320,32,128,128),
            inception(832,384,182,384,48,128,128),
            nn.AvgPool2d(2)
        )
        
        self.classifier = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [7]:
test_net = googlenet(3, 10, True)
test_x = Variable(torch.zeros(1, 3, 96, 96))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

block 1 output: torch.Size([1, 64, 23, 23])
block 2 output: torch.Size([1, 192, 11, 11])
block 3 output: torch.Size([1, 480, 5, 5])
block 4 output: torch.Size([1, 832, 2, 2])
block 5 output: torch.Size([1, 1024, 1, 1])
output: torch.Size([1, 10])


In [8]:
# 准备数据
def data_tf(x):
    x = x.resize((96, 96), 2)
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x)
    return x

train_set = CIFAR10('../data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_set = CIFAR10('../data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)


    
    

In [9]:
# 开始训练
from C5_CNN.utils import train
net = googlenet(3, 10)
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()
train(net, train_data, test_data, 10, optimizer, criterion)

/home/rong/PycharmProjects/PyTorchTutorals/C5_CNN/utils.py:52: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
/home/rong/PycharmProjects/PyTorchTutorals/C5_CNN/utils.py:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 1.423303, Train Acc: 0.485845, Valid Loss: 1.231334, Valid Acc: 0.553514, Time 00:01:42
Epoch 1. Train Loss: 0.916792, Train Acc: 0.680562, Valid Loss: 0.880567, Valid Acc: 0.688998, Time 00:01:53
Epoch 2. Train Loss: 0.695142, Train Acc: 0.762076, Valid Loss: 0.854800, Valid Acc: 0.714157, Time 00:01:55
Epoch 3. Train Loss: 0.566685, Train Acc: 0.806422, Valid Loss: 0.931036, Valid Acc: 0.700080, Time 00:01:49
Epoch 4. Train Loss: 0.472933, Train Acc: 0.838272, Valid Loss: 0.739834, Valid Acc: 0.756090, Time 00:01:49
Epoch 5. Train Loss: 0.394990, Train Acc: 0.862904, Valid Loss: 0.581834, Valid Acc: 0.808806, Time 00:01:49
Epoch 6. Train Loss: 0.336377, Train Acc: 0.884417, Valid Loss: 0.721201, Valid Acc: 0.775759, Time 00:01:49
Epoch 7. Train Loss: 0.274238, Train Acc: 0.905250, Valid Loss: 0.583532, Valid Acc: 0.815695, Time 00:01:49
Epoch 8. Train Loss: 0.237678, Train Acc: 0.917087, Valid Loss: 1.024969, Valid Acc: 0.724341, Time 00:01:52
Epoch 9. Train Loss